In [1]:
#@title Bibliotecas padrões
import pandas as pd
import pandas.io.sql as sqlio
import sys
#import psycopg2
from ydata_profiling import ProfileReport
import numpy as nps
from botocore import UNSIGNED
from botocore.config import Config
from urllib.parse import quote_plus
from pyathena import connect
import boto3
import boto3.session
from botocore import exceptions
import plotly.express as px

#Carregando as bibliotecas necessárias para a análise estatística
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 8)
import statsmodels.api as sm
import statsmodels.tsa as tsa
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_process import arma_generate_sample
from scipy.stats import *

import warnings
# Suprimir todos os warnings
warnings.filterwarnings("ignore")

/home/gabriel/anaconda3/envs/.conda/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/gabriel/.local/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [2]:
class CustomException(Exception):
    pass

json_manifest_dbt = "target/manifest.json"
athena_bucket = "todos-athena-us-east-1"
athena_tmp_folder = f"s3://{athena_bucket}/"


In [3]:
def execute_athena_sql(query):
    client = boto3.client('athena', region_name='us-east-1')
    queryStart = client.start_query_execution(
    QueryString = query,
    ResultConfiguration = { 'OutputLocation': athena_tmp_folder})
    queryExecution = client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])
    while queryExecution['QueryExecution']['Status']['State'] in ('RUNNING', 'QUEUED'):
        time.sleep(5)
        queryExecution = client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])

In [4]:
def execute_athena_query(query):
    cursor = connect(s3_staging_dir=athena_tmp_folder,
                    region_name="us-east-1").cursor()
    cursor.execute(query)
    colls=','.join(str(f"{e[0]}") for e in cursor.description)
    results=pd.DataFrame(list(cursor), columns=colls.split(","))
    return results

In [7]:
sql = '''
select * from todos_data_lake_trusted_feegow.form_9872_amostra 
'''

In [16]:
df = execute_athena_query(sql)

In [9]:
df

,id,cpf_paciente,nome_paciente,paciente_id,tipo_informacao,conteudo_resumo_json,nome_profissional,profissional_id,documento_conselho,data_hora,is_active
0,70592085,52664961772,Jose Das Gracas,10164694,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Caio Mauro Tei...",Jose Leandro Magalhaes,225178.0,51492,2023-10-04 08:53:23,1
1,73590477,70139341609,Erik Castro Dantas,67870375,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Erik Castro Da...",Giovanna Amabile Xavier Borges Dorazio,512378.0,97555,2023-12-07 10:14:58,0
2,70561963,39873916857,Giovana Guedes Dea,58265006,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Giovana Guedes...",Adriana Nunes Da Silva,499326.0,25567,2023-10-03 14:31:05,0
3,70961119,07507217825,Angela Maria Moura Gomes,67297570,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Angela Maria M...",José Alberto Iasbech,509574.0,62833,2023-10-11 13:03:36,0
4,70729091,33002398420,Jose Lustosa Da Silveira,62945084,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Jose Lustosa D...",Pierre Bruno Pinto De Sousa,493192.0,0897,2023-10-06 10:47:12,0
...,...,...,...,...,...,...,...,...,...,...,...
429,73359267,03278642490,João Gutenberg Ferreira Cabral,60886673,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: João Gutenberg...",Vanessa Rocha Sérvulo,512791.0,12508,2023-12-01 18:41:47,0
430,71729724,08000303892,Cleuza Maria Helena Da Silva Vicente,16282088,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Cleuza Maria H...",Roberto Rodrigues Junior,511387.0,133172,2023-10-27 14:15:08,0
431,71801192,71013882482,Hannah Witorya Lucas Borges,64055467,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Hannah Witorya...",Milton Takeo Ito,252420.0,65334,2023-10-30 09:56:35,1
432,72567945,62801988049,MARIA ALICE DA ROSA VARGAS,59893731,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: MARIA ALICE DA...",Sergio Ramos Vieira,240264.0,17960,2023-11-15 12:40:05,1


In [17]:
import json
from pandas import json_normalize

# Converter a coluna 'conteudo_resumo_json' para objetos Python
df['conteudo_resumo_json'] = df['conteudo_resumo_json'].apply(json.loads)

# Normalizar a coluna JSON
df_normalized = json_normalize(df['conteudo_resumo_json'])

# Concatenar o DataFrame original com o DataFrame normalizado
df = pd.concat([df, df_normalized], axis=1)

# Remover a coluna original com JSON
df = df.drop('conteudo_resumo_json', axis=1)

In [18]:
df

,id,cpf_paciente,nome_paciente,paciente_id,tipo_informacao,nome_profissional,profissional_id,documento_conselho,data_hora,is_active,Dados do Paciente,Apresentou algum sintoma nos últimos 14 dias?,Quais?,,Tempo:,"Se outra, especificar:","Se sim, quais:"
0,70308440,55592643004,Santa Edeni Pereira Domingues,14453054,Ficha de Avaliação de Risco Cirúrgico,João Marcelo Ramachiotti Soares Amaro,513106.0,51214,2023-09-27 17:42:41,0,Nome: Santa Edeni Pereira Domingues CPF: ...,Sim,,Nome: João Marcelo Ramachiotti Soares Amaro | ...,,,
1,72974550,47378913553,ADÃO RIBEIRO GUIMARAES,13074540,Ficha de Avaliação de Risco Cirúrgico,Guilherme Baladi,510008.0,189430,2023-11-24 09:44:43,0,Nome: Adão Ribeiro Guimaraes CPF: 473789...,Sim,,Nome: Guilherme Baladi | Documento Conselho: 1...,,,
2,72559556,09109391525,Bernardo Oliveira Menezes,64901303,Ficha de Avaliação de Risco Cirúrgico,Flavia Alesandra Diaz Garces,510658.0,235797,2023-11-15 10:10:53,0,Nome: Bernardo Oliveira Menezes CPF: 091...,Sim,,Nome: Flavia Alesandra Diaz Garces | Documento...,,,
3,70429280,42117720820,Maria Jose Borin Da Paixao,14832458,Ficha de Avaliação de Risco Cirúrgico,Michelly Motta Morais Bayerl,132485.0,10545,2023-09-29 18:04:12,0,Nome: Maria Jose Borin Da Paixao CPF: 42...,Sim,,Nome: Michelly Motta Morais Bayerl | Documento...,,,
4,71873932,00776452606,Rosa Felix De Araujo,13031528,Ficha de Avaliação de Risco Cirúrgico,Philipe Cruz Braga,235975.0,40906,2023-10-31 11:01:28,0,Nome: Rosa Felix De Araujo CPF: 00776452...,Sim,,Nome: Philipe Cruz Braga | Documento Conselho:...,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,73478315,41519272049,Rejane Andrade Godinho,16682399,Ficha de Avaliação de Risco Cirúrgico,Sergio Ramos Vieira,240264.0,17960,2023-12-05 12:55:37,1,Nome: Rejane Andrade Godinho CPF: 415192...,Não,,Nome: Sergio Ramos Vieira | Documento Conselho...,,,"LOSARTANA, SELOZOK, FUROSEMIDA"
430,73023088,01496248910,Solange Lima Arnais Rogasleski,67686873,Ficha de Avaliação de Risco Cirúrgico,Angelo Vicente Tamborelli Torre,497369.0,29302,2023-11-25 08:14:47,1,Nome: Solange Lima Arnais Rogasleski CPF:...,Não,,Nome: Angelo Vicente Tamborelli Torre | Docume...,,,
431,70196680,60867396768,Jose Soares De Moraes,66571070,Ficha de Avaliação de Risco Cirúrgico,Osmar Gonçalves Dias Júnior,166884.0,5201094211,2023-09-26 09:31:32,0,Nome: Jose Soares De Moraes CPF: 6086739...,Sim,,Nome: Osmar Gonçalves Dias Júnior | Documento ...,,,
432,72959777,02908125935,Maria Aparecida Oliveira Marcolino,60720868,Ficha de Avaliação de Risco Cirúrgico,Fernanda Brígido Castilhos,495738.0,31236,2023-11-23 18:05:14,0,Nome: Maria Aparecida Oliveira Marcolino ...,Sim,,Nome: Fernanda Brígido Castilhos | Documento C...,,,


In [27]:
df = df.loc[df['cpf_paciente'] == '62801988049']
df

,id,cpf_paciente,nome_paciente,paciente_id,tipo_informacao,nome_profissional,profissional_id,documento_conselho,data_hora,is_active,Dados do Paciente,Apresentou algum sintoma nos últimos 14 dias?,Quais?,Texto,Tempo:,"Se outra, especificar:","Se sim, quais:"
301,72567945,62801988049,MARIA ALICE DA ROSA VARGAS,59893731,Ficha de Avaliação de Risco Cirúrgico,Sergio Ramos Vieira,240264.0,17960,2023-11-15 12:40:05,1,Nome: MARIA ALICE DA ROSA VARGAS CPF: 62...,Sim,,Nome: Sergio Ramos Vieira | Documento Conselho...,,PATOLOGIA RENAL,ATENOLOL 100 MG DIA


In [10]:
#RETIRAR CARACTRES ESPECIAIS E SALVAR EM ARQUIVO JSON
from unidecode import unidecode

df_cleaned = df.copy()

df_cleaned.columns = df_cleaned.columns.str.replace('[^a-zA-Z0-9]', '_')

def clean_column_values(column):
    return column.apply(lambda x: unidecode(''.join(e for e in str(x) if e.isalnum() or e.isspace())))

df_cleaned = df_cleaned.apply(clean_column_values)

# Salvar o DataFrame limpo em um arquivo JSON
df_cleaned.to_json('dados_limpos.json', orient='records', lines=True)

In [ ]:
#pip install unidecode

In [24]:
nome_paciente_procurado = 'Aldo Bentes Vieira'
df = df.loc[df['nome_paciente'] == nome_paciente_procurado]
df

,id,cpf_paciente,nome_paciente,paciente_id,tipo_informacao,conteudo_resumo_json,nome_profissional,profissional_id,documento_conselho,data_hora,is_active
189,73357334,21491429291,Aldo Bentes Vieira,66784431,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Aldo Bentes Vi...",Luiz Otávio De Souza Carvalho,494269.0,2280,2023-12-01 17:38:53,1


In [34]:

import json

j = json.loads('{"Dados do Paciente" : " Nome:  Aldo Bentes Vieira    CPF:  21491429291    Data de Nascimento:  02/09/1963    Gênero:  Masculino", "Apresentou algum sintoma nos últimos 14 dias?" : "Não", "Quais?" : " ", "1" : " ", "1" : "Hipertensão", "Tempo:" : " ", "1" : " ", "Tempo:" : " ", "1" : "Diabete Mellitus", "Tempo:" : " ", "1" : " ", "Tempo:" : " ", "1" : " ", "Tempo:" : " ", "1" : " ", "Se outra, especificar:" : " ", "Tempo:" : " ", "1" : " ", "1" : "Não", "Se sim, quais:" : " ", "1" : "Não", "Se sim, quais:" : " ", "1" : "Sim", "Se sim, quais:" : "LOSARTANA E GLIFAGE XR", "1" : "  P.A: 114 X 73 HGT: F.C: 82 Peso:  ", "1" : "PACIENTE APRESENTA RISCO CIRURGICO CARDIOLÓGICO CLASSE I DE GOLDMAN, OU SEJA, BAIXO RISCO PARA EVENTOS CARDÍACOS GRAVES ( LIBERADO PARA O PROCEDIMENTO ", "1" : "Nome: Luiz Otávio De Souza Carvalho | Documento Conselho: 2280 | Data Atendimento: 01/12/2023"}')
j[''] = '12'
j


{'Dados do Paciente': ' Nome: \xa0Aldo Bentes Vieira    CPF: \xa021491429291    Data de Nascimento:  02/09/1963    Gênero:\xa0 Masculino',
 'Apresentou algum sintoma nos últimos 14 dias?': 'Não',
 'Quais?': ' ',
 '1': 'Nome: Luiz Otávio De Souza Carvalho | Documento Conselho: 2280 | Data Atendimento: 01/12/2023',
 'Tempo:': ' ',
 'Se outra, especificar:': ' ',
 'Se sim, quais:': 'LOSARTANA E GLIFAGE XR',
 '': '12'}

In [21]:
df = execute_athena_query(sql)

In [22]:
import pandas as pd

# Função para substituir "" por "1" em uma string
def substituir_valor(conteudo):
    return conteudo.replace('""', '"Texto"')

# Aplicar a função à coluna conteudo_resumo_json
df['conteudo_resumo_json'] = df['conteudo_resumo_json'].apply(lambda x: substituir_valor(x))

# Exibir o DataFrame atualizado
df


,id,cpf_paciente,nome_paciente,paciente_id,tipo_informacao,conteudo_resumo_json,nome_profissional,profissional_id,documento_conselho,data_hora,is_active
0,70592085,52664961772,Jose Das Gracas,10164694,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Caio Mauro Tei...",Jose Leandro Magalhaes,225178.0,51492,2023-10-04 08:53:23,1
1,73590477,70139341609,Erik Castro Dantas,67870375,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Erik Castro Da...",Giovanna Amabile Xavier Borges Dorazio,512378.0,97555,2023-12-07 10:14:58,0
2,70561963,39873916857,Giovana Guedes Dea,58265006,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Giovana Guedes...",Adriana Nunes Da Silva,499326.0,25567,2023-10-03 14:31:05,0
3,70961119,07507217825,Angela Maria Moura Gomes,67297570,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Angela Maria M...",José Alberto Iasbech,509574.0,62833,2023-10-11 13:03:36,0
4,70729091,33002398420,Jose Lustosa Da Silveira,62945084,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Jose Lustosa D...",Pierre Bruno Pinto De Sousa,493192.0,0897,2023-10-06 10:47:12,0
...,...,...,...,...,...,...,...,...,...,...,...
429,72581411,81234350610,Zelita Rodrigues Pereira De Morais,67476483,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Zelita Rodrigu...",Roberta Mara Batista Lima,256139.0,76263,2023-11-16 09:13:11,1
430,72715777,10696144913,Luana Kubinski Francisco,67721162,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Luana Kubinski...",Angelo Vicente Tamborelli Torre,497369.0,29302,2023-11-18 12:12:05,1
431,73141972,04758237964,Amabile Suellen De Almeida De Oliveira,12953841,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Amabile Suelle...",Tana Mara Casemiro Belinati Loureiro,247918.0,7478,2023-11-28 11:14:15,0
432,73431563,69726000815,Deoclecio Paioli Simoes,18426400,Ficha de Avaliação de Risco Cirúrgico,"{""Dados do Paciente"" : "" Nome: Deoclecio Paio...",André De Freitas,238991.0,76694,2023-12-04 15:28:47,0


In [23]:
import json
from pandas import json_normalize

# Converter a coluna 'conteudo_resumo_json' para objetos Python
df['conteudo_resumo_json'] = df['conteudo_resumo_json'].apply(json.loads)

# Normalizar a coluna JSON
df_normalized = json_normalize(df['conteudo_resumo_json'])

# Concatenar o DataFrame original com o DataFrame normalizado
df = pd.concat([df, df_normalized], axis=1)

# Remover a coluna original com JSON
df = df.drop('conteudo_resumo_json', axis=1)

In [24]:
df

,id,cpf_paciente,nome_paciente,paciente_id,tipo_informacao,nome_profissional,profissional_id,documento_conselho,data_hora,is_active,Dados do Paciente,Apresentou algum sintoma nos últimos 14 dias?,Quais?,Texto,Tempo:,"Se outra, especificar:","Se sim, quais:"
0,70592085,52664961772,Jose Das Gracas,10164694,Ficha de Avaliação de Risco Cirúrgico,Jose Leandro Magalhaes,225178.0,51492,2023-10-04 08:53:23,1,"Nome: Caio Mauro Teixeira Lima Altura: 1,7...",Não,,Nome: Jose Leandro Magalhaes | Documento Conse...,,,
1,73590477,70139341609,Erik Castro Dantas,67870375,Ficha de Avaliação de Risco Cirúrgico,Giovanna Amabile Xavier Borges Dorazio,512378.0,97555,2023-12-07 10:14:58,0,Nome: Erik Castro Dantas CPF: 7013934160...,Sim,,Nome: Giovanna Amabile Xavier Borges Dorazio |...,,,
2,70561963,39873916857,Giovana Guedes Dea,58265006,Ficha de Avaliação de Risco Cirúrgico,Adriana Nunes Da Silva,499326.0,25567,2023-10-03 14:31:05,0,Nome: Giovana Guedes Dea CPF: 3987391685...,Sim,,Nome: Adriana Nunes Da Silva | Documento Conse...,,,
3,70961119,07507217825,Angela Maria Moura Gomes,67297570,Ficha de Avaliação de Risco Cirúrgico,José Alberto Iasbech,509574.0,62833,2023-10-11 13:03:36,0,Nome: Angela Maria Moura Gomes CPF: 0750...,Sim,,Nome: José Alberto Iasbech | Documento Conselh...,,,
4,70729091,33002398420,Jose Lustosa Da Silveira,62945084,Ficha de Avaliação de Risco Cirúrgico,Pierre Bruno Pinto De Sousa,493192.0,0897,2023-10-06 10:47:12,0,Nome: Jose Lustosa Da Silveira CPF: 3300...,Sim,,Nome: Pierre Bruno Pinto De Sousa | Documento ...,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,72581411,81234350610,Zelita Rodrigues Pereira De Morais,67476483,Ficha de Avaliação de Risco Cirúrgico,Roberta Mara Batista Lima,256139.0,76263,2023-11-16 09:13:11,1,Nome: Zelita Rodrigues Pereira De Morais ...,Sim,,Nome: Roberta Mara Batista Lima | Documento Co...,,,"Insulina, Anlodipino, Metformina"
430,72715777,10696144913,Luana Kubinski Francisco,67721162,Ficha de Avaliação de Risco Cirúrgico,Angelo Vicente Tamborelli Torre,497369.0,29302,2023-11-18 12:12:05,1,Nome: Luana Kubinski Francisco CPF: 1069...,Não,,Nome: Angelo Vicente Tamborelli Torre | Docume...,,,
431,73141972,04758237964,Amabile Suellen De Almeida De Oliveira,12953841,Ficha de Avaliação de Risco Cirúrgico,Tana Mara Casemiro Belinati Loureiro,247918.0,7478,2023-11-28 11:14:15,0,Nome: Amabile Suellen De Almeida De Oliveira...,Sim,,Nome: Tana Mara Casemiro Belinati Loureiro | D...,,,
432,73431563,69726000815,Deoclecio Paioli Simoes,18426400,Ficha de Avaliação de Risco Cirúrgico,André De Freitas,238991.0,76694,2023-12-04 15:28:47,0,Nome: Deoclecio Paioli Simoes CPF: 69726...,Sim,,Nome: André De Freitas | Documento Conselho: 7...,,,


In [26]:
#exportar para csv e testar
# Caminho do arquivo CSV
caminho_arquivo_csv = 'form_9872_amostra_202401241037.csv'

# Exportar DataFrame para CSV
df.to_csv(caminho_arquivo_csv, index=False)

In [47]:
df['conteudo_resumo_json'] = df['conteudo_resumo_json'].apply(json.loads)

KeyError: 'conteudo_resumo_json'

In [46]:
import json
import pandas as pd

# Remover espaços em branco antes de aplicar json.loads
df['conteudo_resumo_json'] = df['conteudo_resumo_json'].str.strip()

# Converter a coluna 'conteudo_resumo_json' para objetos Python
df['conteudo_resumo_json'] = df['conteudo_resumo_json'].apply(json.loads)

# Normalizar a coluna JSON
df_normalized = json_normalize(df['conteudo_resumo_json'])

# Concatenar o DataFrame original com o DataFrame normalizado
df = pd.concat([df, df_normalized], axis=1)

# Remover a coluna original com JSON
df = df.drop('conteudo_resumo_json', axis=1)

df


KeyError: 'conteudo_resumo_json'

In [60]:
import pandas as pd
import json

# Seu JSON
json_data = '''
{"Dados do Paciente" : " Nome:  Aldo Bentes Vieira    CPF:  21491429291    Data de Nascimento:  02/09/1963    Gênero:  Masculino", 
"Apresentou algum sintoma nos últimos 14 dias?" : "Não", 
"Quais?" : " ", 
"Texto" : " ", 
"Texto" : "Hipertensão", 
"Tempo:" : " ", 
"Texto" : " ", 
"Tempo:" : " ", 
"Texto" : "Diabete Mellitus", 
"Tempo:" : " ", 
"Texto" : " ", 
"Tempo:" : " ", 
"Texto" : " ", 
"Tempo:" : " ", 
"Texto" : " ", 
"Se outra, especificar:" : " ", 
"Tempo:" : " ", 
"Texto" : " ", 
"Texto" : "Não", 
"Se sim, quais:" : " ", 
"Texto" : "Não", "Se sim, quais:" : " ", 
"Texto" : "Sim", 
"Se sim, quais:" : "LOSARTANA E GLIFAGE XR", 
"Texto" : "  P.A: 114 X 73 HGT: F.C: 82 Peso:  ", 
"Texto" : "PACIENTE APRESENTA RISCO CIRURGICO CARDIOLÓGICO CLASSE I DE GOLDMAN, OU SEJA, BAIXO RISCO PARA EVENTOS CARDÍACOS GRAVES ( LIBERADO PARA O PROCEDIMENTO ", 
"Texto" : "Nome: Luiz Otávio De Souza Carvalho | Documento Conselho: 2280 | Data Atendimento: 01/12/2023"}
'''

# Carregar JSON
data = json.loads(json_data)

# Renomear chaves duplicadas
for key in data.keys():
    if list(data.values()).count(data[key]) > 1:
        data[key] = key + "_dup"

# Converter para DataFrame
df = pd.json_normalize(data)

# Exibir DataFrame
df


,Dados do Paciente,Apresentou algum sintoma nos últimos 14 dias?,Quais?,Texto,Tempo:,"Se outra, especificar:","Se sim, quais:"
0,Nome: Aldo Bentes Vieira CPF: 2149142929...,Não,Quais?_dup,Nome: Luiz Otávio De Souza Carvalho | Document...,Tempo:_dup,,LOSARTANA E GLIFAGE XR


In [61]:
df

,Dados do Paciente,Apresentou algum sintoma nos últimos 14 dias?,Quais?,Texto,Tempo:,"Se outra, especificar:","Se sim, quais:"
0,Nome: Aldo Bentes Vieira CPF: 2149142929...,Não,Quais?_dup,Nome: Luiz Otávio De Souza Carvalho | Document...,Tempo:_dup,,LOSARTANA E GLIFAGE XR
